In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate()

Activating environment at `~/Documents/study-validated-numerics/notebook/Project.toml`


# Chapter 2. Interval Arithmetic
## 2.4. Floating Point Interval Arithmetic
### 2.4.1. A ~~MATLAB~~ Julia Implementation of Interval Arithmetic

In [2]:
struct Interval{T<:Real}
    lo::T
    hi::T
    function Interval(lo::T, hi::T) where T<:Real
        lo <= hi ? new{T}(lo, hi) : throw(ArgumentError("The endpoints do not define an interval."))
    end
end

Interval(x::Real) = Interval(x, x)

Main.##421.Interval

In [3]:
Interval(1.0, 2.0)

Main.##421.Interval{Float64}(1.0, 2.0)

In [4]:
Interval(1f0, 2f0)

Main.##421.Interval{Float32}(1.0f0, 2.0f0)

In [5]:
Interval(1, 2)

Main.##421.Interval{Int64}(1, 2)

In [6]:
Interval(1.0)

Main.##421.Interval{Float64}(1.0, 1.0)

In [7]:
Interval(1.0, 1.0)

Main.##421.Interval{Float64}(1.0, 1.0)

In [8]:
# NG
# Interval(2.0, 1.0)

Define `display`

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*